<a href="https://colab.research.google.com/github/Alfred9/Natural-Language-Processing-Projects/blob/main/Topic%20Modelling/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install sparknlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=20e2c5cb0b611ac3e5a8bfea45b5631ad260ffc48e5216257c8052e3050f1ccf
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

spark = sparknlp.start()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df =pd.read_csv("drive/My Drive/Datasets/CommentsApril2017.csv")
df.head()

<ipython-input-5-319e1484a83e>:3: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv("drive/My Drive/Datasets/CommentsApril2017.csv")


,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,...,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,...,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,...,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,...,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,...,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


In [ ]:
df = spark.read.format("csv").option("header",True).load("drive/My Drive/Datasets/CommentsApril2017.csv")
df.count()

243866

Lets set up an NLP pipeline for text preprocessing. It begins by assembling raw text data, tokenizing it into individual words, and normalizing them. Stop words are then removed, and the resulting clean tokens are finalized for further analysis. Finally, the pipeline is applied to the input DataFrame, and a limited subset of tokenized data is extracted for subsequent processing.

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("commentBody") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

finisher = Finisher() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

nlp_pipeline = Pipeline(
    stages=[document_assembler,
            tokenizer,
            normalizer,
            stopwords_cleaner,
            finisher])

nlp_model = nlp_pipeline.fit(df)

processed_df  = nlp_model.transform(df)

tokens_df = processed_df.select('tokens').limit(10000)

Lets employs the CountVectorizer algorithm to convert tokenized text data into numerical feature vectors suitable for machine learning. It initializes the CountVectorizer object with parameters specifying input and output columns, as well as constraints on vocabulary size and minimum document frequency. The model is then fitted to the tokenized DataFrame to learn the vocabulary, and subsequently transforms the tokenized data into feature vectors. The resulting DataFrame contains both the original tokens and their corresponding numerical representations, facilitating downstream machine learning tasks.

In [ ]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=1000, minDF=3.0)

cv_model = cv.fit(tokens_df)

vectorized_tokens = cv_model.transform(tokens_df)

### LDA (Latent Dirichlet Allocation) model to generate clusters of a selected number of topics:

Latent Dirichlet Allocation (LDA) is a probabilistic generative model used for topic modeling, a technique that identifies abstract topics within a collection of documents.

We utilize Latent Dirichlet Allocation (LDA), a topic modeling algorithm, to uncover latent topics within a collection of text data. It initializes an LDA model with parameters specifying the number of topics and maximum iterations. The model is then fitted to the vectorized tokens obtained from the preceding CountVectorizer step. Through this process, LDA identifies underlying topics present in the text corpus, providing insights into the thematic structure of the data.

In [ ]:
from pyspark.ml.clustering import LDA

num_topics = 15
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(vectorized_tokens)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

vocab = cv_model.vocabulary
topics = model.describeTopics()
topics_rdd = topics.rdd
topics_words = topics_rdd \
       .map(lambda row: row['termIndices']) \
       .map(lambda idx_list: [vocab[idx] for idx in idx_list]) \
       .collect()

def get_words(idx_list):
    return [vocab[idx] for idx in idx_list]

udf_get_words = udf(get_words, ArrayType(StringType()))
topics = topics.withColumn("words", udf_get_words(topics.termIndices))

topics_df = topics.select("topic", "words")

topics_df.show(truncate=False)

+-----+------------------------------------------------------------------------------------+
|topic|words                                                                               |
+-----+------------------------------------------------------------------------------------+
|0    |[like, people, think, looks, man, candy, cotton, see, Vancouver, city]              |
|1    |[Quinn, Dar, game, Carrie, gets, Putin, got, kill, good, back]                      |
|2    |[population, live, likely, close, many, Ive, wanted, love, look, drive]             |
|3    |[Trump, people, like, women, Fox, never, many, get, dont, men]                      |
|4    |[public, programs, student, government, success, costs, every, male, world, benefit]|
|5    |[people, get, man, many, see, know, like, country, money, look]                     |
|6    |[men, lots, women, small, son, role, room, company, goes, good]                     |
|7    |[Trump, believe, people, getting, episode, still, head, get, ev